<a href="https://colab.research.google.com/github/fcela/chr/blob/master/examples/CHR_extended.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Uncomment the line below if you want to install additional R dependencies
# needed for BART and Quantile Regression Forest
#
#!R -e "install.packages('BART', 'quantregForest')"
!pip install git+https://github.com/fcela/scikit-garden
!pip install git+https://github.com/fcela/chr

  Cloning https://github.com/fcela/scikit-garden to /tmp/pip-req-build-bnr78r_r
  Running command git clone -q https://github.com/fcela/scikit-garden /tmp/pip-req-build-bnr78r_r
  Created wheel for scikit-garden: filename=scikit_garden-0.1.3-cp37-cp37m-linux_x86_64.whl size=1006350 sha256=2cd03f32276522f47a14f6a0c5ef2b6c7136c0753bc80724c1aed63a2837f914
  Stored in directory: /tmp/pip-ephem-wheel-cache-6981s269/wheels/08/84/9a/f9188194893215c75af7e551fd6152642c93944bae0146f1c4
Successfully built scikit-garden
  Cloning https://github.com/fcela/chr to /tmp/pip-req-build-lo9mgel_
  Running command git clone -q https://github.com/fcela/chr /tmp/pip-req-build-lo9mgel_
  Created wheel for chr: filename=chr-0.0.1.dev16-py3-none-any.whl size=18821 sha256=6d1566591cb8ff78bcb80387899f0cf7b36a4039b17c04a6cc23962985577e3e
  Stored in directory: /tmp/pip-ephem-wheel-cache-rqignje_/wheels/3b/cd/41/ddf95531a0aa539ed81ad7e53a9b00e54eb2655cc80d6d673d
Successfully built chr


In [2]:
import os
import sys
import pdb
import torch

print("Is CUDA available? {}".format(torch.cuda.is_available()))

Is CUDA available? True


In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from tqdm.autonotebook import tqdm
from sklearn.model_selection import train_test_split

## Generating synthetic data

In [4]:
def plot_func(x, y, quantiles=None, quantile_labels=None, max_show=5000,
              shade_color="", method_name="", title="", filename=None, save_figures=False):
    
    """ Scatter plot of (x,y) points along with the constructed prediction interval 
    
    Parameters
    ----------
    x : numpy array, corresponding to the feature of each of the n samples
    y : numpy array, target response variable (length n)
    quantiles : numpy array, the estimated prediction. It may be the conditional mean,
                or low and high conditional quantiles.
    shade_color : string, desired color of the prediciton interval
    method_name : string, name of the method
    title : string, the title of the figure
    filename : sting, name of the file to save the figure
    save_figures : boolean, save the figure (True) or not (False)
    
    """
    
    x_ = x[:max_show]
    y_ = y[:max_show]
    if quantiles is not None:
        quantiles = quantiles[:max_show]
    
    fig = plt.figure()
    inds = np.argsort(np.squeeze(x_))
    plt.plot(x_[inds], y_[inds], 'k.', alpha=.2, markersize=10, fillstyle='none')
    
    if quantiles is not None:
        num_quantiles = quantiles.shape[1]
    else:
        num_quantiles = 0  
    
    if quantile_labels is None:
        pred_labels = ["NA"] * num_quantiles
    for k in range(num_quantiles):
        label_txt = 'Quantile {q}'.format(q=quantile_labels[k])
        plt.plot(x_[inds], quantiles[inds,k], '-', lw=2, alpha=0.75, label=label_txt)
    
    plt.ylim([-2, 20])
    plt.xlabel('$X$')
    plt.ylabel('$Y$')
    plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left')
    plt.title(title)
    if save_figures and (filename is not None):
        plt.savefig(filename, bbox_inches='tight', dpi=300)
    
    plt.show()

## Split-conformal calibration

Split-conformal with pre-trained black box

In [5]:
import pandas as pd
data = pd.read_csv("https://drive.google.com/u/0/uc?id=1BV2kV0b5O9u2E6dg8nJ-fXImH7wEcIV3&export=download")
data

,Median_House_Value,Median_Income,Median_Age,Tot_Rooms,Tot_Bedrooms,Population,Households,Latitude,Longitude,Distance_to_coast,Distance_to_LA,Distance_to_SanDiego,Distance_to_SanJose,Distance_to_SanFrancisco
0,452600.0,8.3252,41,880,129,322,126,37.88,-122.23,9263.040773,556529.158342,735501.806984,67432.517001,21250.213767
1,358500.0,8.3014,21,7099,1106,2401,1138,37.86,-122.22,10225.733072,554279.850069,733236.884360,65049.908574,20880.600400
2,352100.0,7.2574,52,1467,190,496,177,37.85,-122.24,8259.085109,554610.717069,733525.682937,64867.289833,18811.487450
3,341300.0,5.6431,52,1274,235,558,219,37.85,-122.25,7768.086571,555194.266086,734095.290744,65287.138412,18031.047568
4,342200.0,3.8462,52,1627,280,565,259,37.85,-122.25,7768.086571,555194.266086,734095.290744,65287.138412,18031.047568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,78100.0,1.5603,25,1665,374,845,330,39.48,-121.09,162031.481121,654530.186299,830631.543047,248510.058162,222619.890417
20636,77100.0,2.5568,18,697,150,356,114,39.49,-121.21,160445.433537,659747.068444,836245.915229,246849.888948,218314.424634
20637,92300.0,1.7000,17,2254,485,1007,433,39.43,-121.22,153754.341182,654042.214020,830699.573163,240172.220489,212097.936232
20638,84700.0,1.8672,18,1860,409,741,349,39.43,-121.32,152005.022239,657698.007703,834672.461887,238193.865909,207923.199166


In [6]:
Y_data = data.Median_House_Value.values
X_data = data.copy()
X_data = X_data.drop(columns="Median_House_Value").to_numpy()

In [7]:
import numpy as np
X_data_mu = np.mean(X_data, axis=0)
X_data_sd = np.std(X_data, axis=0)
X_data_normalized = (X_data - X_data_mu) / X_data_sd

Y_data_mu = np.mean(Y_data)
Y_data_sd = np.std(Y_data)
Y_data_normalized = (Y_data - Y_data_mu) / Y_data_sd


## Fit an estimator for the quantiles

QNet will fit a neural network to estimate quantiles, with the following architecture:

```
nn.Sequential(
    nn.Linear(num_features, num_hidden),
    nn.ReLU(),
    nn.Dropout(dropout),
    nn.Linear(num_hidden, num_hidden),
    nn.ReLU(),
    nn.Dropout(dropout),
    nn.Linear(num_hidden, num_quantiles),
)
```

(Expand code below for details)



In [ ]:
#@title
# This is the base implementation of the methods in the original paper, for
# ease of access and to enable modifications as needed
#
# As it stands, it is equivalent to:
#
# --> from chr.black_boxes import QNet, QRF


import torch
import torch.nn as nn
import torch.optim as optim
#import torch.tensor as tensor
from torch import Tensor as tensor
from torch.utils.data import DataLoader

import six
import sys
sys.modules['sklearn.externals.six'] = six

from skgarden import RandomForestQuantileRegressor

from sklearn.model_selection import train_test_split
from tqdm.autonotebook import tqdm
import numpy as np

from chr.utils import RegressionDataset

import warnings

import pdb

class NNet(nn.Module):
    """ Conditional quantile estimator, formulated as neural net
    """
    def __init__(self, quantiles, num_features, num_hidden=64, dropout=0.1, no_crossing=False):
        """ Initialization
        Parameters
        ----------
        quantiles : numpy array of quantile levels (q), each in the range (0,1)
        num_features : integer, input signal dimension (p)
        num_hidden : integer, hidden layer dimension
        dropout : float, dropout rate
        no_crossing: boolean, whether to explicitly prevent quantile crossovers
        """
        super(NNet, self).__init__()

        self.no_crossing = no_crossing

        self.num_quantiles = len(quantiles)

        # Construct base network
        self.base_model = nn.Sequential(
            nn.Linear(num_features, num_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(num_hidden, num_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(num_hidden, self.num_quantiles),
        )
        self.init_weights()

    def init_weights(self):
        """ Initialize the network parameters
        """
        for m in self.base_model:
            if isinstance(m, nn.Linear):
                nn.init.orthogonal_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        """ Run forward pass
        """
        x = self.base_model(x)
        if self.no_crossing:
            y,_ = torch.sort(x,1)
        else:
            y = x
        return y


class AllQuantileLoss(nn.Module):
    """ Pinball loss function
    """
    def __init__(self, quantiles):
        """ Initialize
        Parameters
        ----------
        quantiles : pytorch vector of quantile levels, each in the range (0,1)
        """
        super().__init__()
        self.quantiles = quantiles

    def forward(self, preds, target):
        """ Compute the pinball loss
        Parameters
        ----------
        preds : pytorch tensor of estimated labels (n)
        target : pytorch tensor of true labels (n)
        Returns
        -------
        loss : cost function value
        """
        #assert not target.requires_grad
        #assert preds.size(0) == target.size(0)

        errors = target.unsqueeze(1)-preds
        Q = self.quantiles.unsqueeze(0)
        loss = torch.max((Q-1.0)*errors, Q*errors).mean()

        return loss


class QNet:
    """ Fit a neural network (conditional quantile) to training data
    """
    def __init__(self, quantiles, num_features, no_crossing=False, dropout=0.2, learning_rate=0.001,
                 num_epochs=100, batch_size=16, num_hidden=64, random_state=0, calibrate=0, verbose=False):
        """ Initialization
        Parameters
        ----------
        quantiles : numpy array of quantile levels (q), each in the range (0,1)
        num_features : integer, input signal dimension (p)
        learning_rate : learning rate
        random_state : integer, seed used in CV when splitting to train-test
        """

        # Detect whether CUDA is available
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        # Store input (sort the quantiles)
        quantiles = np.sort(quantiles)
        self.quantiles = torch.from_numpy(quantiles).float().to(self.device)
        self.num_features = num_features

        # Define NNet model
        self.model = NNet(self.quantiles, self.num_features, num_hidden=num_hidden, dropout=dropout, no_crossing=no_crossing)
        self.model.to(self.device)

        # Initialize optimizer
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

        # Initialize loss function
        self.loss_func = AllQuantileLoss(self.quantiles)

        # Store variables
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.random_state = random_state
        self.calibrate = int(calibrate)

        # Initialize training logs
        self.loss_history = []
        self.test_loss_history = []
        self.full_loss_history = []

        # Validation
        self.val_period = 10

        self.verbose = verbose

    def fit(self, X, Y, return_loss=False):
        Y = Y.flatten().astype(np.float32)
        X = X.astype(np.float32)
        
        dataset = RegressionDataset(X, Y)
        num_epochs = self.num_epochs
        if self.calibrate>0:
            # Train with 80% of samples
            n_valid = int(np.round(0.2*X.shape[0]))
            loss_stats = []
            for b in range(self.calibrate):
                X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=n_valid, random_state=self.random_state+b)
                train_dataset = RegressionDataset(X_train, Y_train)
                val_dataset = RegressionDataset(X_valid, Y_valid)
                loss_stats_tmp = self._fit(train_dataset, num_epochs, val_dataset=val_dataset)
                loss_stats.append([loss_stats_tmp['val']])                
                # Reset model
                self.model.init_weights()

            loss_stats = np.matrix(np.concatenate(loss_stats,0)).T

            loss_stats = np.median(loss_stats,1).flatten()
            # Find optimal number of epochs
            num_epochs = self.val_period*(np.argmin(loss_stats)+1)
            loss_stats_cal = loss_stats

        # Train with all samples
        loss_stats = self._fit(dataset, num_epochs)
        if self.calibrate:
            loss_stats = loss_stats_cal

        if return_loss:
          return loss_stats

    def _fit(self, train_dataset, num_epochs, val_dataset=None):
        batch_size = self.batch_size

        # Initialize data loaders
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size)
        if val_dataset is not None:
            val_loader = DataLoader(dataset=val_dataset, batch_size=1)

        num_samples, num_features = train_dataset.X_data.shape
        print("Training with {} samples and {} features.". \
              format(num_samples, num_features))

        loss_stats = {'train': [], "val": []}

        X_train_batch = train_dataset.X_data.to(self.device)
        y_train_batch = train_dataset.y_data.to(self.device)

        for e in tqdm(range(1, num_epochs+1)):

            # TRAINING
            train_epoch_loss = 0
            self.model.train()

            if batch_size<500:
            
              for X_train_batch, y_train_batch in train_loader:
                  X_train_batch, y_train_batch = X_train_batch.to(self.device), y_train_batch.to(self.device)
                  self.optimizer.zero_grad()

                  y_train_pred = self.model(X_train_batch).to(self.device)

                  train_loss = self.loss_func(y_train_pred, y_train_batch)

                  train_loss.backward()
                  self.optimizer.step()

                  train_epoch_loss += train_loss.item()

            else:
                self.optimizer.zero_grad()

                y_train_pred = self.model(X_train_batch).to(self.device)

                train_loss = self.loss_func(y_train_pred, y_train_batch)

                train_loss.backward()
                self.optimizer.step()

                train_epoch_loss += train_loss.item()

            # VALIDATION
            if val_dataset is not None:
                if e % self.val_period == 0:
                    self.model.eval()
                    with torch.no_grad():
                        val_epoch_loss = 0
                        for X_val_batch, y_val_batch in val_loader:
                            X_val_batch, y_val_batch = X_val_batch.to(self.device), y_val_batch.to(self.device)
                            y_val_pred = self.model(X_val_batch).to(self.device)
                            val_loss = self.loss_func(y_val_pred, y_val_batch)
                            val_epoch_loss += val_loss.item()

                    loss_stats['val'].append(val_epoch_loss/len(val_loader))
                    self.model.train()

            else:
                loss_stats['val'].append(0)

            if e % self.val_period == 0:
                loss_stats['train'].append(train_epoch_loss/len(train_loader))

            if (e % 10 == 0) and (self.verbose):
                if val_dataset is not None:
                    print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | ', end='')
                    print(f'Val Loss: {val_epoch_loss/len(val_loader):.5f} | ', flush=True)
                else:
                    print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | ', flush=True)

        return loss_stats

    def predict(self, X):
        """ Estimate the label given the features
        Parameters
        ----------
        x : numpy array of training features (nXp)
        Returns
        -------
        ret_val : numpy array of predicted labels (n)
        """
        self.model.eval()
        ret_val = self.model(torch.from_numpy(X).to(self.device).float().requires_grad_(False))
        return ret_val.cpu().detach().numpy()

    def get_quantiles(self):
        return self.quantiles.cpu().numpy()
    

class QRF:
    """ Fit a random forest (conditional quantile) to training data
    """
    def __init__(self, quantiles, min_samples_leaf=5, n_estimators=100, n_jobs=1,
                 random_state=0, verbose=False):
        """ Initialization
        Parameters
        ----------
        quantiles : numpy array of quantile levels (q), each in the range (0,1)
        num_features : integer, input signal dimension (p)
        random_state : integer, seed used in quantile random forests
        """

        self.device = 'cpu'
        # Store input (sort the quantiles)
        self.quantiles = torch.from_numpy(np.sort(quantiles)).float().to(self.device) 
        # Define RF model
        self.model = RandomForestQuantileRegressor(random_state=random_state,
                                                   min_samples_leaf=min_samples_leaf,
                                                   n_estimators=n_estimators,
                                                   n_jobs=n_jobs, verbose=verbose)

    def fit(self, X, Y, return_loss=None):
        warnings.filterwarnings("ignore", category=FutureWarning)
        self.model.fit(X, Y)
        warnings.filterwarnings("default", category=FutureWarning)
        return 0

    def predict(self, X):
        """ Estimate the label given the features
        Parameters
        ----------
        x : numpy array of training features (nXp)
        Returns
        -------
        ret_val : numpy array of predicted labels (n)
        """
        quantiles = self.quantiles.cpu()
        ret_val = np.zeros((X.shape[0], len(quantiles)))
        print("Predicting RF quantiles:")
        for i in tqdm(range(len(quantiles))):
            ret_val[:,i] = self.model.predict(X,quantile=100*quantiles[i])
        return ret_val
    
    def get_quantiles(self):
        return self.quantiles.cpu().numpy()


In [9]:
from chr.methods import CHR

method = "QNet"
  
# Split the data
X_train, X_calib_test, Y_train, Y_calib_test = train_test_split(X_data_normalized, Y_data_normalized, test_size=0.5, random_state=2020)
X_calib, X_test, Y_calib, Y_test = train_test_split(X_calib_test, Y_calib_test, test_size=0.5, random_state=2020)
grid_quantiles = np.arange(0.01,1.0,0.01)


if method == "QNet":

  bbox = QNet(grid_quantiles, num_features=X_train.shape[1], no_crossing=True, batch_size=1000, dropout=0.1, 
              num_epochs=10000, learning_rate=0.0005, num_hidden=256, calibrate=0)
  
elif method == "QRF":
  bbox = QRF(grid_quantiles, n_estimators=100, min_samples_leaf=50, random_state=2020)

bbox.fit(X_train, Y_train)

Training with 10320 samples and 13 features.


  0%|          | 0/10000 [00:00<?, ?it/s]

In [10]:
# Initialize and calibrate the new method
chr = CHR(bbox, ymin=-3, ymax=3, y_steps=200, delta_alpha=0.001, randomize=True)
chr.calibrate(X_calib, Y_calib, alpha=0.01)

Computing conformity scores.


  0%|          | 0/999 [00:00<?, ?it/s]

Calibrated alpha (nominal level: 0.01): 0.009.


0.009

In [11]:
bands = chr.predict(X_test)
bands


array([[-0.85929648,  2.87939698],
       [-3.        ,  0.73869347],
       [-0.1959799 ,  3.        ],
       ...,
       [-3.        ,  0.04522613],
       [ 0.04522613,  3.        ],
       [-3.        , -0.34673367]])

In [14]:
# Compute what % of observations fall outside the interval
np.mean( (Y_test > bands[:,1]) | (Y_test < bands[:,0] ) )

0.009689922480620155

In [15]:
# Compute all the quantiles
q_hat = bbox.predict(X_test)

In [46]:
Y_test_hat = (q_hat[:,grid_quantiles == 0.5]).flatten()

In [48]:
err = np.abs(Y_test - Y_test_hat)

In [60]:
iqr = bands[:,1] - bands[:,0]

# note we have a few cases where the bounds actually cross ... we need to
# study this
np.sum(bands[:,0]>bands[:,1])



21

In [55]:
# Show top places where we have high error and tight bounds -- i.e. we are 
# overconfident

XX = pd.DataFrame(X_test)
XX.columns = data.drop(columns="Median_House_Value").columns.values
XX['err'] = err
XX['iqr'] = iqr
XX['norm_err'] = err/iqr
XX.sort_values(by=['norm_err'], ascending=False)

,Median_Income,Median_Age,Tot_Rooms,Tot_Bedrooms,Population,Households,Latitude,Longitude,Distance_to_coast,Distance_to_LA,Distance_to_SanDiego,Distance_to_SanJose,Distance_to_SanFrancisco,err,iqr,norm_err
1753,4.542288,-1.719432,-1.153193,-1.227093,-1.214619,-1.235978,0.940184,-1.452618,-0.777712,1.141063,1.147120,-1.323133,-1.484061,1.828728,2.050251,0.891953
1734,5.858286,1.856182,-1.107355,-1.200980,-1.210204,-1.238594,0.729500,-1.008392,-0.050892,0.776117,0.837329,-1.463860,-1.151714,2.328491,2.683417,0.867733
4593,-0.245066,1.299975,1.967048,1.709491,2.904824,1.740582,1.019775,-1.447627,-0.773009,1.193206,1.193243,-1.270627,-1.525025,2.961271,3.768844,0.785724
2944,0.417012,-0.289187,-0.547206,-0.346356,-0.700682,-0.299591,-0.876380,0.688653,-0.767500,-0.954890,-0.855929,0.785737,0.803941,2.204992,2.984925,0.738709
2219,0.230042,0.187562,-0.340473,-0.208666,-0.535551,-0.192351,-0.754652,0.593818,-0.587605,-1.034849,-0.740155,0.631582,0.670212,2.244337,3.105528,0.722691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554,-0.800079,-1.322142,-0.310678,-0.400957,-0.486100,-0.380675,1.960830,-1.008392,3.020509,1.765582,1.678177,-0.330052,-0.590874,0.157835,-2.954774,-0.053417
3555,-0.190849,-0.448103,0.075742,-0.073351,-0.094908,-0.119114,2.288561,-1.377748,2.248910,2.160692,2.018554,0.031892,-0.324726,0.180681,-3.135678,-0.057621
2573,-0.615636,-1.639974,0.314562,0.634089,0.237120,0.717880,1.417734,-0.888600,0.567942,1.265435,1.254994,-0.895315,-1.000328,0.172824,-2.773869,-0.062304
1042,-0.599319,-0.050812,-0.727352,-0.719067,-0.636219,-0.762553,0.570316,-1.108218,-0.792722,0.717268,0.781610,-1.355947,-1.076834,0.288228,-2.201005,-0.130953


### Now identify slices where performance is particularly bad

(coming soon)